# 📌 자막 데이터 크롤링  
* urllib 라이브러리 사용해 파일 다운로드

### ✏️ 애니메이션 자막 페이지 목록 받아오기

In [ ]:
# 라이브러리 읽어들이기
import requests
import re
import os
from bs4 import BeautifulSoup as bs

# 파일 주소 크롤링
def bs_parsing(url):
  response = requests.get(root_url + url)
  soup = bs(response.text, "html.parser")
  elements = soup.select("table#flisttable td > a")
  
  # 주소 담을 리스트
  page_list = []
  
  # 상세 페이지 주소 리스트화
  for (idx, el) in enumerate(elements, 1):
    page_list.append(root_url + el.attrs["href"].replace("%2F", "/"))
    
  return page_list

# 리스트 값을 txt 파일에 한 줄씩 저장
def save_text_file(page_list):
  f = open("C:/Users/sodud/study/ssafy_spec_pjt/subtitle/data/page_url_list.txt", "w", encoding="UTF8")
  for page in page_list:
    f.write(page)
    f.write("\n")
  f.close
  ("[SUCCESS] 자막 페이지 목록 저장")

# 자막 사이트
root_url = "https://kitsunekko.net"

# 일본어 자막 목록 페이지
page_url = "/dirlist.php?dir=subtitles/japanese/"

# 자막 페이지 목록 리스트로 저장
page_list = bs_parsing(page_url)

# 리스트를 txt 파일로 저장
save_text_file(page_list)

### ✏️ srt 확장자 파일만 다운로드

In [ ]:
import requests
import re
import os
from bs4 import BeautifulSoup as bs
from inspect import getfile
from urllib import request
from urllib.error import HTTPError

def read_text_file(page_list_file, start, unit):
  flag = True
  f = open(page_list_file, "r", encoding="UTF8")
  lines = f.readlines()[start: start+unit]
  
  if len(lines) == 0:
    return
  
  if len(lines) < unit:
    flag = False
  
  for line in lines:
    
    # 주소가 아니면 pass
    if line[-3:] != "%2F":
      pass
    
    response = requests.get(line[:-1])
    soup = bs(response.text, "html.parser")
    files = soup.select("table#flisttable td > a")
    
    # 파일 주소 담을 리스트
    file_urls = []
    
    # 파일 읽기
    for f in files:
      file_path= f.attrs["href"]
      
      # srt 파일만 저장
      if file_path[-3:] == "srt":
        file_urls.append("/" + file_path)
      else:
        pass
    
    # 자막 페이지 확인
    print(file_path)
    
    # 파일이 존재하는지 검색
    file_search(file_urls)
    
  f.close
  
  if flag == False:
    return
  
  return read_text_file(page_list_file, start + unit, unit)

# 파일이 존재하는지 검색
def file_search(file_urls):
  for file_url in file_urls:
    # 파일 이름만 추출
    depth = file_url.split("/")
    filename = depth[len(depth)-1]
    try:
      # 파일 저장 경로
      path = "C:\\Users\\sodud\\study\\ssafy_spec_pjt\\subtitle\\data\\text\\"
      
      # 파일이 저장되어 있지 않으면 다운로드
      if os.path.isfile(path + filename):
        print("이미 존재하는 파일: " + filename)
      else:
        get_download(root_url + file_url.replace(" ", "%20"), filename, path)
        
    except HTTPError as err:
      print("다운로드 중 에러 발생: " + filename + "/" + err)
      pass

# 파일 다운로드
"""
[*]   url: 파일 다운로드 경로
[*] fname: 저장할 파일명
[*]   dir: 저장 폴더 경로
"""
def get_download(url, fname, dir):
  try:
    os.chdir(dir)
    # 다운로드 요청
    request.urlretrieve(url, fname)
    print("다운로드 완료: " + fname)
  except HTTPError as err:
    print("에러 발생:" + err)
    return

# 자막 사이트
root_url = "https://kitsunekko.net"

# 자막 파일 페이지 목록 파일
page_list_file = "C:/Users/sodud/study/ssafy_spec_pjt/subtitle/data/page_url_list.txt"

# 목록 파일 읽기
read_text_file(page_list_file, 0, 10)